# GasBotty Demonstration Notebook

Authors: Anonymous Authors

---

In this work, we focus on the **accurate detection** and **reading** of gas prices, and their **contextual association** to gas grade and payment type (**Mult-Metric Extraction** *in the Wild*). In this notebook, we demonstrate the **GasBotty predictor**, a composite neural network model, by applying it to one of five dfferent sample images.

### Setup Requirements

This predictor builds off of the work of two existing frameworks, the [Keras RetinaNet](https://github.com/fizyr/keras-retinanet) and [DeepLabV3](https://github.com/leimao/DeepLab_v3) and has the following dependencies, listed within `requirements.txt`:
* cython
* keras-resnet
* h5py
* keras
* matplotlib
* numpy
* opencv-python
* pillow
* progressbar2
* tensorflow
* torch
* torchvision
* scikit-learn
* argparse
* glob2

### Install Requirements

In [ ]:
!pip install -r ./requirements.txt

### Import Dependencies & GasBotty Module

In [ ]:
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import numpy as np
import glob
import os
os.chdir('./GasBotty/')

from GasBotty.gasbotty import *

### Install the Keras RetinaNet Depencies Locally

This codebase contains a previously cloned copy of the Keras RetinaNet repository and need only be installed.

In [ ]:
!pip install .

### Setup Keras RetinaNet

Note: The cloned Keras RetinaNet must be re-setup. Depending on your environment configuration, use either `python` or `python3`:

`python3 setup.py build_ext --inplace`

In [ ]:
!python3 setup.py build_ext --inplace

### Download each of the Released Models



In [ ]:
%%bash

wget https://github.com/anon93642/GasBotty/releases/download/v1.0/weights_101.pt -P ./weights/
wget https://github.com/anon93642/GasBotty/releases/download/v1.0/resnet101_LABELSG1.h5 -P ./weights/
wget https://github.com/anon93642/GasBotty/releases/download/v1.0/resnet101_DIGITSFINAL.h5 -P ./weights/
wget https://github.com/anon93642/GasBotty/releases/download/v1.0/priceresnet50_csv_1all.h5 -P ./weights/

### Step 1: Import each of pre-trained models

GasBotty is a composite neural network incorporating four component models; after downloading them in the previous step, we load each here.

In [ ]:
sign_model, price_model, digit_model, label_model  = load_models()

### Step 2: Load an Example Image

By changing the index, an alternative image can be selected. Choose one of: `img_idx` \\( \in [1,2,3,4,5] \\). The ground truth (`gt_file`) dataframe is loaded in the final step to compare against the predicted results.

In [ ]:
img_idx  = 1
img_file = f'../example-images/example_{img_idx}.png'
gt_file  = f'../example-images/example_{img_idx}.csv' 
image    = cv2.imread(img_file)
plt.imshow(cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB))
plt.show()

### Step 3: From the Image, Predict the Sign-Level Mask

In [ ]:
mask = get_mask(image, sign_model)
plt.imshow(mask)
plt.show()

### Step 4: From the Predicted Sign-Level Mask, Extract  the Border

In [ ]:
border = get_border(mask)
plt.imshow(border)
plt.show()

### Step 5: Using the Extracted Border, Detect the Hough Lines

In [ ]:
line_df, line_image = get_lines(border, image.copy())
plt.imshow(cv2.cvtColor(line_image.copy(), cv2.COLOR_BGR2RGB))
plt.show()   

### Step 6: From the Detected Hough lines, Obtain the Points of Intersection

In [ ]:
intersection_df, intersection_image = get_intersections(line_df, line_image)
plt.imshow(cv2.cvtColor(intersection_image.copy(), cv2.COLOR_BGR2RGB))
plt.show()  

### Step 7: From the Points of Intersection, Obtain the Sign-Level Corners

In [ ]:
src_pts, corners_image = get_corners(intersection_df, intersection_image)
plt.imshow(cv2.cvtColor(corners_image.copy(), cv2.COLOR_BGR2RGB))
plt.show() 

### Step 8: Keystone Correct the Sign using Four Corners

This steep generates a sign-level, perspective-corrected image.

In [ ]:
keystone_image = get_KS(src_pts, image.copy())
plt.imshow(cv2.cvtColor(keystone_image.copy(), cv2.COLOR_BGR2RGB))
plt.show() 

### Step 9: Using the Sign-Level Image, Extract all Prices

In [ ]:
where_prices, price_image = price_level(keystone_image.copy(), price_model)
plt.imshow(cv2.cvtColor(price_image.copy(), cv2.COLOR_BGR2RGB))
plt.show() 

### Step 10: Using the Sign-Level Image, Extract all Labels

In [ ]:
where_labels, label_image = label_level(keystone_image.copy(), label_model, price_image)
plt.imshow(cv2.cvtColor(label_image.copy(), cv2.COLOR_BGR2RGB))
plt.show() 

### Step 11: Using all Extracted Price-Level Images, Detect all Digits

In [ ]:
prices = []
for price in where_prices:
    current_price, price_image = read_digits(keystone_image.copy()[price[1]:price[3], price[0]:price[2]] , digit_model )
    prices.append(( price[0], price[1], price[2], price[3], current_price))        
    plt.imshow(cv2.cvtColor(price_image.copy(), cv2.COLOR_BGR2RGB))
    plt.title(str(current_price))
    plt.show() 

### Step 12: Using all Extracted Prices & Labels, Associate & Generate Final Prediction

In [ ]:
df , associate_image = associate(prices, where_labels, label_image)
plt.imshow(cv2.cvtColor(associate_image.copy(), cv2.COLOR_BGR2RGB))
plt.show() 
print(f'Complete Predicted Dataframe:\n{df}')

### Final Step: Evaluate Performance with Ground Truth Annotation

In [ ]:
df_groundtruth = pd.read_csv(gt_file)
df_groundtruth['Price'] = df_groundtruth['Price'].round(3)
print(f'All-or-Nothing Accuracy (ANA): {ANA(df, df_groundtruth)}\n\nPredicted:\n{df}\n\nGround Truth:\n{df_groundtruth}')

### Thats all Folks!